In [68]:
class Vertex():
    WHITE = 0
    GREY = 1
    BLACK = 2    
    
    def __init__(self, label=None, color=WHITE):
        self.color = color
        self.label = label
        self.predecessor = None
        self.discovered = None
        self.finished = None
        self.topo_next = None

    def __repr__(self):
        return repr(self.__dict__)
        
class Graph():
    def __init__(self, V=[], E=[]):
        A = [ [] for _ in range(len(V)) ]
        for src, tgt in E:
            A[src].append(tgt)
            
        self.vertices = [ Vertex(label=v) for v in V ]
        self.adj = A
        self.topo_next = None

    def __repr__(self):
        return repr(self.__dict__)

import graphviz
def graph_to_dot(G, title=None):
    dot = graphviz.Digraph(comment=title)
    
    # Add nodes
    for idx, vertex in enumerate(G.vertices):
        #dot.attr('node', fontcolor='white')
        #dot.attr('node', style='filled', fillcolor=colors[cluster_id])
        dot.node(str(idx), str(vertex.label), shape='circle')

    # Add edges
    for src, adjacents in enumerate(G.adj):
        for tgt in adjacents:
            dot.edge(str(src), str(tgt))

In [75]:
def _dfs_visit(G, vertex_idx):
    # Mark discovered
    #print('discovered', vertex_idx)
    G.time += 1
    v = G.vertices[vertex_idx]
    v.discovered = G.time
    v.color = Vertex.GREY
    
    # Search edges
    for a_idx in G.adj[vertex_idx]:
        a = G.vertices[a_idx]
        if a.color == Vertex.WHITE:
            a.predecessor = vertex_idx
            _dfs_visit(G, a_idx)

    # Mark finished
    #print('done with', vertex_idx)
    v.color = Vertex.BLACK
    G.time += 1
    v.finished = G.time

    # Update topological sort
    v.topo_next = G.topo_next
    G.topo_next = vertex_idx
    
    
def dfs(G):
    # Reset
    G.time = 0
    for vertex in G.vertices:
        vertex.color = Vertex.WHITE
        vertex.predecessor = None
    # Search
    for idx, vertex in enumerate(G.vertices):
        if vertex.color == Vertex.WHITE:
            _dfs_visit(G, idx)

def is_ancestor(G, u, v):
    # u is ancestor of v if and only if, d[u], f[u] contains d[v],f[v]
    V = G.vertices[v]
    U = G.vertices[u]
    assert U.finished is not None
    assert V.finished is not None
    print("[{},{}] [{},{}]".format(V.discovered, V.finished, U.discovered, U.finished))
    return V.discovered > U.discovered and V.finished < U.finished

def is_decendant(G, u, v):
    """u is decentant of v if and only if, d[u], f[u] is subinterval of d[v],f[v]"""
    V = G.vertices[v]
    U = G.vertices[u]
    assert U.finished is not None
    assert V.finished is not None
    return U.discovered > V.discovered and U.finished < V.finished

def is_unrelated(G, u, v):
    """u is unrelated to v if and only if, d[u], f[u] and d[v], f[v] are disjoint intervals"""
    V = G.vertices[v]
    U = G.vertices[u]
    assert U.finished is not None
    assert V.finished is not None
    below = U.discovered < V.discovered and U.finished < V.finished 
    above = U.discovered > V.discovered and U.finished > V.finished
    return above or below

    return dot

def topological_sorted(G):
    topo_sorted = []
    v = G.topo_next
    while v is not None:
        topo_sorted.append(v)
        v = v.topo_next
    return topo_sorted

def is_descending(l):
    descending = ( c > n for c, n in zip(l, l[1:]) )
    return all(descending)

def is_acyclic(G):
    return False

def test_dfs():
    simple = Graph(
        ( 0, 1, 2, 3 ),
        [ (0,3), (1,2), (2,0), ],
    )
    dfs(simple)
    assert is_decendant(simple, 3, 0)
    assert is_ancestor(simple, 0, 3)
    # fails because during dfs we start at 0, which goes to 3, and then ends
    # the longer chain from 1->2 is not connected to 0 because it has already been finished
    #assert is_decendant(simple, 0, 1)
    #assert is_ancestor(simple, 1, 0)
    
    longer = Graph(
        ( 0, 1, 2, 3, 4, 5 ),
        [ (0,1), (1,2), (2,3), (5,4) ],
    )
    dfs(longer)
    assert is_decendant(longer, 3, 0)
    assert is_ancestor(longer, 0, 3)
    assert is_decendant(longer, 2, 0)
    assert is_ancestor(longer, 0, 2)
    assert is_unrelated(longer, 3, 4)
    
    ts = topological_sorted(longer)
    assert is_descending([v.finished for v in ts])

    # TODO: detect loops / DAG
    # by edge labling
    # Forward, backward, cross, tree
    #return graph_to_dot(simple)
    #return 'tests passed'
    
test_dfs()


[2,3] [1,4]
[4,5] [1,8]
[3,6] [1,8]


In [ ]:

# Articulation point
# a vertex is an articulation point if its removal disconnects G
# Biconnected graph
# a graph which has no articulation points
# that is, to disconnect, must remove at least two vertices
# Biconnected component
# maximally biconnected subgraph of G
